In [11]:
import numpy as np
from numba import njit, vectorize
from BPnumba.GeneticOperators import CrossOX
from numba.typed import List as NumbaList


In [14]:


chromosome = NumbaList([1,2,3,4,5,6])
chromosomeR = NumbaList([0,0,0,0,0,0])
chromosome2 = NumbaList([2,5,1,6,3,4])
chromosomeR = NumbaList([0,0 ,0,0,0,0])

CrossOX(chromosome, chromosome2,chromosomeR,chromosomeR,  2, 4)

[array([1, 6, 3, 4, 5, 2], dtype=int64),
 array([0, 0, 0, 0, 0, 0], dtype=int64)]